# SAM 3 with Linear Probing on DVARF Dataset

This notebook demonstrates the complete pipeline for training and applying a linear probe on top of SAM 3's zero-shot predictions to improve object detection performance on the DVARF aerial accident dataset.

## 1. Environment Setup

Clone the repository and install required dependencies.

In [ ]:
%cd /content

# Clone the DVARF repository
!git clone https://github.com/AntoFratta/DVARF.git

%cd /content/DVARF

# Create a Colab-compatible requirements file by removing Windows-specific packages
!grep -v "triton-windows" requirements.txt > requirements_colab.txt

# Install all project dependencies
!pip install -r requirements_colab.txt

## 2. Extract Dataset

Extract the data archive containing images and annotations.

In [ ]:
%cd /content/DVARF

# Install unrar utility
!apt-get update -y > /dev/null
!apt-get install -y unrar > /dev/null

# Extract data.rar to create the data/ directory structure
!unrar x data.rar ./ > /dev/null

# Verify extraction
print("After extracting data.rar:")
!ls
print("\nInside data/:")
!ls data
print("\nInside data/raw/:")
!ls data/raw
print("\nImage splits:")
!ls data/raw/images

## 3. Install SAM 3

Clone and install the official SAM 3 repository.

In [ ]:
%cd /content

# Clone the official SAM 3 repository from Meta
!git clone https://github.com/facebookresearch/sam3.git

In [ ]:
%cd /content/sam3

# Install SAM 3 in editable mode
!pip install -e .

## 4. Hugging Face Authentication

Authenticate with Hugging Face to access SAM 3 model weights.

In [ ]:
from huggingface_hub import login

# Login to Hugging Face (widget will prompt for your token)
login()

## 5. Run SAM 3 on Training Split

Execute zero-shot inference on the training set to generate predictions for linear probe training.

In [ ]:
%cd /content/DVARF

import sys
if "/content/DVARF" not in sys.path:
    sys.path.insert(0, "/content/DVARF")

from scripts.run_sam3_on_split import run_sam3_on_split

# Run SAM 3 on TRAIN split (segmentations not needed for linear probing)
run_sam3_on_split(
    split="train",
    score_threshold=0.26,
    max_images=None,
    save_segmentations=False,
    max_masks_per_image_per_class=None,
)

## 6. Build Linear Probe Dataset

Create training dataset for the linear probe by matching predictions with ground truth annotations.

In [ ]:
%cd /content/DVARF

# Build the linear probe training dataset by matching SAM 3 predictions to ground truth
!python scripts/build_linear_probe_dataset.py

## 7. Train Linear Probe

Train a simple logistic regression classifier for each object class to refine SAM 3's predictions.

In [ ]:
%cd /content/DVARF

# Train the linear probe classifier
!python scripts/train_linear_probe.py

## 8. Run SAM 3 on Test Split

Execute zero-shot inference on the test set.

In [ ]:
%cd /content/DVARF

import sys
if "/content/DVARF" not in sys.path:
    sys.path.insert(0, "/content/DVARF")

from scripts.run_sam3_on_split import run_sam3_on_split

# Run SAM 3 on TEST split with segmentation masks saved
run_sam3_on_split(
    split="test",
    score_threshold=0.26,
    max_images=None,
    save_segmentations=True,
    max_masks_per_image_per_class=None,
)

## 9. Apply Linear Probe to Test Predictions

Refine the test set predictions using the trained linear probe.

In [ ]:
%cd /content/DVARF

# Apply the trained linear probe to test predictions
!python scripts/apply_linear_probe_to_split.py

## 10. Evaluate Linear Probe Results

Compute metrics on the refined predictions.

In [ ]:
%cd /content/DVARF

# Evaluate the linear probe enhanced predictions and save metrics
!python scripts/eval_sam3_linear_probe_on_split.py | tee results/sam3_linear_probe_test_metrics.txt

## 11. Archive Results

Create compressed archives of prediction outputs and training artifacts for download.

In [ ]:
%cd /content/DVARF

# Archive zero-shot SAM 3 predictions on test set
!zip -r results/sam3_test_predictions.zip data/processed/predictions/sam3_yolo/test

# Archive linear probe enhanced predictions on test set
!zip -r results/sam3_linear_probe_test_predictions.zip data/processed/predictions/sam3_linear_probe_yolo/test

In [ ]:
%cd /content/DVARF

# Archive linear probe training artifacts (dataset and weights)
!zip -r results/sam3_linear_probe_training_artifacts.zip data/processed/linear_probe